In [ ]:
! pip install transformers
! pip install lime
! pip install eli5
! pip install datasets

     |████████████████████████████████| 4.0 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 6.6 MB 38.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 275 kB 9.1 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=3add71dd584df39e54c0c620f1ed604da4a4fb3e1c8320df27f0b7322b38314f
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
     |████████████████████████████████| 106 kB 7.6 MB/s 
     |████████████████████████████████| 325 kB 7.7 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 212 kB 51.9 MB/

In [ ]:
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler
import torch

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from eli5.lime._vectorizer import SingleDocumentVectorizer
from sklearn.base import clone, BaseEstimator
from eli5.lime.utils import (
    fit_proba,
    score_with_sample_weight,
    mean_kl_divergence,
    fix_multiclass_predict_proba,
    rbf
)
from eli5.lime.lime import _train_local_classifier

from transformers import BertTokenizer, BertForTokenClassification,DistilBertTokenizer
from torch.nn.functional import softmax
import numpy as np
import datasets
from datasets import load_dataset 
import re
from itertools import groupby
from operator import itemgetter

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
# example = ["Sweet dreams are made of these, who am I to disagree, I travel the World, everybody is looking for Something"]
example = ["Clifford Jackson"]
ner_results = nlp(example)
print(ner_results)


# tokenizer(example)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[[{'entity': 'B-PER', 'score': 0.99967015, 'index': 1, 'word': 'Clifford', 'start': 0, 'end': 8}, {'entity': 'I-PER', 'score': 0.99953246, 'index': 2, 'word': 'Jackson', 'start': 9, 'end': 16}]]


### Issue: 

Some of the masked word will be splitted into sub-wrods by the tokenizer, therefore I am trying to use a simpler tokenizer instead. But If there is other ways......

**method simple**: Just toss samples with different length after tokenization

**method hard**: write a new sampler that takes a sequence of strings (tokenizer.tokenize) and output a string with correct length.

**method mid**: find the position of subword and padd short samples with [unk] on that position.

In [ ]:
#old method
# text = "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something"
text = "Sweet dreams are made of these, who am I to disagree, I travel the World, everybody is looking for Something"

class EntitySampler_OLD(MaskingTextSampler):
  def __init__(self,
                token_pattern=None,  # type: Optional[str]
                bow=True,            # type: bool
                random_state=None,
                replacement='',      # type: str
                min_replace=1,       # type: Union[int, float]
                max_replace=1.0,     # type: Union[int, float]
                group_size=1,        # type: int
                entity_pos = None,
                entity_word = None,
                 ):
    super().__init__(token_pattern, bow, random_state, replacement, min_replace, max_replace, group_size)
    self.entity_pos = entity_pos
    self.entity_word = entity_word

  def sample_near(self, doc, n_samples=1):
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    # type: (str, int) -> Tuple[List[str], np.ndarray]
    samples = []
    similarities = []
    doc = " ".join(tokenizer.tokenize(doc))
    tokenlength = len(tokenizer.tokenize(doc))
    maxround = 0
    max_flag = 1
    while len(samples) < n_samples:
      if max_flag:
        temp_min_replace = self.min_replace
        self.min_replace = 0.7
        docs, similarity, mask, text = self.sample_near_with_mask(doc=doc, n_samples=int(n_samples*0.5))
        self.min_replace = temp_min_replace
        max_flag = 0
      else:
        docs, similarity, mask, text = self.sample_near_with_mask(doc=doc, n_samples=int(n_samples*1.5))
      badsamples = []
      for i in range(len(docs)):
        # print(tokenizer.tokenize(docs[i])[self.entity_pos[0]: self.entity_pos[-1]+1 ],self.entity_word)
        tokens = tokenizer.tokenize(docs[i])
        if tokens[self.entity_pos[0]: self.entity_pos[-1]+1 ] != self.entity_word or len(tokens) != tokenlength:
          badsamples.append(i)
      samples += [i for j, i in enumerate(list(docs)) if j not in badsamples]
      similarities += [i for j, i in enumerate(list(similarity)) if j not in badsamples]
      maxround += 1
      if maxround >= 10:
        break

    return samples[0:n_samples], similarities[0:n_samples]

esampler = EntitySampler(
replacement="[UNK]",
token_pattern=None,
bow=False,
entity_pos = [16],
entity_word = ["World"]
# entity_pos = [16,17],
# entity_word = ['U', '##topia']
)
samples, similarity = esampler.sample_near(text, n_samples=10)
samples_worlds= samples
samples

KeyboardInterrupt: ignored

### mtehod mid for smapling

In [ ]:
# text = "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something"
text = "Sweet dreams are made of these, who am I to disagree, I travel the World, everybody is looking for Something"

class EntitySampler(MaskingTextSampler):
  def __init__(self,
                token_pattern=None,  # type: Optional[str]
                bow=True,            # type: bool
                random_state=None,
                replacement='',      # type: str
                min_replace=1,       # type: Union[int, float]
                max_replace=1.0,     # type: Union[int, float]
                group_size=1,        # type: int
                entity_pos = None,
                entity_word = None,
                 ):
    super().__init__(token_pattern, bow, random_state, replacement, min_replace, max_replace, group_size)
    self.entity_pos = entity_pos
    self.entity_word = entity_word

  def sample_near(self, doc, n_samples=1):
    # print("debug sampler")
    # print(doc)
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    r = re.compile("##*")
    # type: (str, int) -> Tuple[List[str], np.ndarray]
    samples = []
    similarities = []
    badsamples = []
    tokens = tokenizer.tokenize(doc)
    doc = " ".join(tokens)
    tokenlength = len(tokens)
    maxround = 0
    max_flag = 1
    taglist = list(filter(r.match, tokens)) # Read Note below
    tagindexes = [tokens.index(item) for item in taglist]
    tagindexes = list(set(tagindexes) - set(self.entity_pos))
    tagindexes.sort()
    tagindexgr = []
    for k, g in groupby(enumerate(tagindexes), lambda x: x[0]-x[1]):
      tagindexgr.append(list(map(itemgetter(1), g)))
    tagindexgr = [  [itm[0]-1] + itm for itm in tagindexgr]
    # print(tagindexgr)

    while len(samples) < n_samples:
      if max_flag:
        temp_min_replace = self.min_replace
        self.min_replace = 0.7
        docs, similarity, mask, text = self.sample_near_with_mask(doc=doc, n_samples=int(n_samples*0.5))
        self.min_replace = temp_min_replace
        max_flag = 0
      else:
        docs, similarity, mask, text = self.sample_near_with_mask(doc=doc, n_samples=int(n_samples*1.5))

      for i in range(len(docs)):
        # print(tokenizer.tokenize(docs[i])[self.entity_pos[0]: self.entity_pos[-1]+1 ],self.entity_word)
        # print(docs[i])
        sp_tokens = docs[i].split(" ")
        if sp_tokens[self.entity_pos[0] : self.entity_pos[-1]+1 ] != self.entity_word or len(sp_tokens) != tokenlength:
          pass
        else:
          # if len(sp_tokens) != tokenlength:
          sp_tokens = ['[UNK]' if tk == "##[UNK]" else tk for tk in sp_tokens]
          for tangr in tagindexgr:
            if any(sp_tokens[itm] == "[UNK]" for itm in tangr):
              for itm in tangr:
                sp_tokens[itm] = "[UNK]"
          samples.append(" ".join(sp_tokens).replace(" ##", ""))
          similarities.append(similarity[i])

      maxround += 1
      if maxround >= 10:
        break

    return samples[0:n_samples], similarities[0:n_samples]

esampler = EntitySampler(
replacement="[UNK]",
token_pattern=None,
bow=False,
entity_pos = [16],
entity_word = ["World"]
# entity_pos = [16,17],
# entity_word = ['U', '##topia']
)
# samples, similarity = esampler.sample_near(text, n_samples=10)
# samples_worlds= samples
# samples

In [ ]:
male = "The homeowner Denali asked the inspector if the house he had purchased was structurally sound."
esampler = EntitySampler(
replacement="[UNK]",
token_pattern=None,
bow=False,
entity_pos = [4, 5],
entity_word = ["Den", "##ali"]
)
samples, similarity = esampler.sample_near(male, n_samples=5)
samples

['[UNK] [UNK] [UNK] [UNK] Denali [UNK] the inspector if the house he had purchased was structurally sound .',
 'The homeowner Denali asked the inspector if the house he had purchased [UNK] structurally sound .',
 'The [UNK] [UNK] [UNK] Denali asked the inspector if the [UNK] [UNK] [UNK] purchased [UNK] structurally sound .',
 'The [UNK] [UNK] [UNK] Denali asked the inspector if the house he [UNK] purchased was [UNK] [UNK] sound .',
 'The [UNK] [UNK] [UNK] Denali asked the inspector [UNK] [UNK] [UNK] he had purchased was [UNK] [UNK] [UNK] .']

In [ ]:
example2 = ["Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something",
            "Sweet dreams are made of these, who am I to disagree, I [UNK] the Utopia, everybody is looking for Something",
            "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, [UNK] is looking for Something"
           ]

# TODO proba = proba of O, ave(Proba of B, proba of I's) if input contain multiple words
class EntityPrediction():
  def __init__(self, posit):
    self.posit = posit

  def pred_proba(self, doc):
    # print("debug pred_proba")
    # print(doc)
    _inputs = tokenizer(doc, return_tensors="pt")
    # print(_inputs["input_ids"])
    _outputs = model(**_inputs)
    _logits = _outputs.logits
    proba = softmax(_logits, dim=2)
    # print("Proba shape:", proba.shape)
    # print("self.pos:", self.posit)
    nmd_proba = None
    if type(self.posit) == int:
      proba = proba[:,self.posit,:]
      B = torch.cat([proba[:, 0:1],proba[:, 1::2]], dim=-1)
      nmd_proba =  B
    elif type(self.posit) == list:
      proba = proba[:,self.posit[0]:self.posit[-1]+1,:]
      # proba = batchsize, sequence len, EntityProb
      B = torch.cat([proba[:, 0:1, 0:1],proba[:, 0:1, 1::2]], dim=-1)
      I = proba[:, 1:, 0::2]
      # print(B.shape, I.shape)
      BI = torch.cat([B,I] , dim=1)
      nmd_proba =  torch.mean(BI, dim = 1)
    nmd_proba = nmd_proba / (torch.sum(nmd_proba, -1).unsqueeze(-1))
    return nmd_proba

# ep = EntityPrediction(posit = [17, 18])
ep = EntityPrediction(posit = 18)
ep.pred_proba(example2)
# ep = EntityPrediction(posit = [17])
# ep.pred_proba(samples_worlds)

tensor([[0.2053, 0.0922, 0.0243, 0.0213, 0.6569],
        [0.9539, 0.0217, 0.0071, 0.0055, 0.0119],
        [0.2138, 0.1000, 0.0288, 0.0277, 0.6297]], grad_fn=<DivBackward0>)

In [ ]:
# text = "Sweet dreams are made of these, who am I to disagree, I travel the World, everybody is looking for Something"

class TestExplainer(TextExplainer):
  def fit(self,doc,predict_proba):
    self.doc_ = doc
    if self.position_dependent:
        samples, sims, mask, text = self.sampler.sample_near_with_mask(
            doc=doc,
            n_samples=self.n_samples
        )
        self.vec_ = SingleDocumentVectorizer(
            token_pattern=self.token_pattern
        ).fit([doc])
        X = ~mask
    else:
        self.vec_ = clone(self.vec).fit([doc])
        # print(doc)
        samples, sims = self.sampler.sample_near(
            doc=doc,
            n_samples=self.n_samples
        )
        X = self.vec_.transform(samples)
        # print("DEBUG: X.shape",X.shape)

    if self.rbf_sigma is not None:
        sims = rbf(1-sims, sigma=self.rbf_sigma)

    self.samples_ = samples
    self.similarity_ = sims
    self.X_ = X
    # print("debug samples")
    # print(samples)
    self.y_proba_ = predict_proba(samples).detach().numpy()
    self.clf_ = clone(self.clf)
    # print(self.samples_ )
    # print("DEBUG: X, sims, proba", X.shape, len(sims), self.y_proba_.shape)
    # global temp_sims, temp_X, temp_proba
    # temp_sims = sims
    # temp_X = X
    # temp_proba = self.y_proba_
    yList = np.argmax(self.y_proba_, axis=1)
    if np.all(yList == yList[0]):
      print("ERROR: unexplainable due to lack of negative samples")
      return None

    self.metrics_ = _train_local_classifier(
        estimator=self.clf_,
        samples=X,
        similarity=sims,
        y_proba=self.y_proba_,
        expand_factor=self.expand_factor,
        random_state=self.rng_
    )
    return self

# esampler = EntitySampler(
# replacement="[UNK]",
# max_replace=2,
# token_pattern=None,
# bow=False,
# entity_pos = [16],
# entity_word = ["World"]
# # entity_pos = [16,17],
# # entity_word = ['U', '##topia']
# )

# te = TestExplainer(
# n_samples=10,
# sampler=esampler,
# # sampler = None,
# # vec = vectorizer,
# position_dependent=False,
# random_state=42
# )

# te.fit(text, ep.pred_proba)

# explain = te.explain_prediction(
# target_names=["O", "MIS", "PER", "ORG", "LOC"],
# top_targets=3
# )
# explain

### Issue:
If the generated samples only contains negative samples or positive samples then we will not get the only one class issue.
Furthermore, this is a limitation of using lime here where the generated sample is super bias, which may cause potential problem.

In [ ]:
# For debug purpose!
from eli5.lime.utils import fit_proba, expanded_X_y_sample_weights, with_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

rng = check_random_state(None)
(X_train, X_test,
  similarity_train, similarity_test,
  y_proba_train, y_proba_test) = train_test_split(temp_X,
                                                  temp_sims,
                                                  temp_proba,
                                                  test_size=0.3,
                                                  random_state=rng)

print(X_train.shape)
print(y_proba_train.shape)

X, y, sample_weight = expanded_X_y_sample_weights(X_train, y_proba_train,
    expand_factor=10,
    sample_weight=similarity_train,
    shuffle=True,
    random_state=rng,
)
fit_params = with_sample_weight(temp_clf, sample_weight, {})

print(y)
temp_clf.fit(X, y, **fit_params)
 
# fit_proba(temp_clf, X_train, y_proba_train,
#           expand_factor=10,
#           sample_weight=similarity_train,
#           random_state=rng)



### find position

In [ ]:
# save index of each entity to dict, TODO: Logic is weird DOUBLE CHECK!

# example = ["Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something",
#            "Sweet dreams are made of these, who am I to disagree, I [UNK] the Utopia, everybody is looking for Something",
#            ]
example = "Sweet dreams are made of these, who am Dong to disagree, I travel the Utopia, Waang is looking for Something"
# example = text

ner_results = nlp(example)
ner_results

def findEntityPos(ner_results):
  # ner_results = nlp(somestring)
  entity_map = {"MIS": [], "PER": [], "ORG": [], "LOC": []}
  ini_list = None
  entype = None
  for item in ner_results:
    if item["entity"] in ["B-MIS","B-PER", "B-ORG","B-LOC"]: # TODO not quite, check the entity type match for B-TYP and Y-TYP
      if ini_list == None:
        ini_list = [item["index"]]
        entype = item["entity"][2:]
      else:
        entity_map[entype].append(ini_list)
        ini_list = [item["index"]]
        entype = item["entity"][2:]
    elif (item["entity"] in ["I-MIS","I-PER", "I-ORG","I-LOC"]) and (ini_list != None) and (entype == item["entity"][2:]):
      ini_list.append(item["index"])
      # not only check is I but also check match entities!
  entity_map[entype].append(ini_list)
  return entity_map

findEntityPos(ner_results)

# count the explainable example 
# dataset = load_dataset('conll2003', split='test')
# # explainable example: 25
# text = " ".join(dataset["tokens"][37])
# print(text)
# nlp(text)

{'LOC': [[17, 18]], 'MIS': [], 'ORG': [], 'PER': [[10], [20]]}

### generate wigogender info

In [ ]:
#cite winogender

def generate(occupation, other_participant, answer, sentence, name, someone=False, context=None):
  toks = sentence.split(" ")
  occ_index = toks.index("$OCCUPATION")
  part_index = toks.index("$PARTICIPANT")
  if answer:
    other_participant = other_participant + " " + name
  elif not answer:
    occupation = occupation + " " + name
  toks[occ_index] = occupation
  if not someone: # we are using the instantiated participant, e.g. "client", "patient", "customer",...
    toks[part_index] = other_participant
  else: # we are using the bleached NP "someone" for the other participant
    # first, remove the token that precedes $PARTICIPANT, i.e. "the"
    toks = toks[:part_index-1]+toks[part_index:]
    # recompute participant index (it should be part_index - 1)
    part_index = toks.index("$PARTICIPANT")
    if part_index == 0:
      toks[part_index] = "Someone"
    else:
      toks[part_index] = "someone"
  NOM = "$NOM_PRONOUN"
  POSS = "$POSS_PRONOUN"
  ACC = "$ACC_PRONOUN"
  special_toks = set({NOM, POSS, ACC})
  female_map = {NOM: "she", POSS: "her", ACC: "her"}
  male_map = {NOM: "he", POSS: "his", ACC: "him"}
  neutral_map = {NOM: "they", POSS: "their", ACC: "them"}
  female_toks = [x if not x in special_toks else female_map[x] for x in toks]
  male_toks = [x if not x in special_toks else male_map[x] for x in toks]
  neutral_toks = [x if not x in special_toks else neutral_map[x] for x in toks]
  male_sent, female_sent, neutral_sent = " ".join(male_toks), " ".join(female_toks), " ".join(neutral_toks)
  neutral_sent = neutral_sent.replace("they was", "they were")
  neutral_sent = neutral_sent.replace("They was", "They were")
  return male_sent, female_sent, neutral_sent

In [ ]:
nmlong = "Denali, Kabecka, Kahun, Kabili, Kadal, Fedyenka, Hihikara, Huemac, Hirarama, Abiah, Adair, Hachi, Amory, Ayden, Lubov, Galina, Aiah, Gennady, Aleksis, Chong, Lyev, Mingyu, Xinyi, Micha, Youki, Flavio, Shiloh, Quill, Leighton, Avery, Aubrey, Micah, Wiley, Marley, Marlow, Harlow, Madison, Ellis, Ellison, Elliott, Arden, Linden, Finley, Emery, Emerson, Ellery, Parker, Remy, Sloane, Mackenzie, Rowan, Sawyer, River, Sage, Cassidy, Remington, Saylor, Skyler, Tate, Tatum, Sutton, Wren, Leslie, Sasha, Monroe, Holland, Hollis, Lennox, Ari, Landry, Rune, Leith, Darcy, Quincy, Darby, Hunter, Dana, Whitney, Ezra, Robin, Bowie, Harper, Emrys, Indy, Indiana, Montana, Dakota, Bennett, Everett, Devon, Alexis, Dominique, Reagan, Stevie, Corey, Salem, Denver, Indigo, Ramsey, Larkin, Greer, Everest, Channing, Lumen, Zephyr, Adair, Bellamy, Cecile, Laken, Percy, Jody, Winslow, Asher, Kit, Camden, Emmett, Spencer"
nmls = nmlong.split(", ")
len(nmls)

113

In [ ]:
# ! wget https://raw.githubusercontent.com/rudinger/winogender-schemas/master/data/all_sentences.tsv
! wget https://raw.githubusercontent.com/rudinger/winogender-schemas/master/data/templates.tsv

import pandas as pd

stcs = pd.read_csv("/content/templates.tsv", sep = '\t')
# index = 0
male_ls = []
female_ls = []
for index, row in stcs.iterrows():
  occupation, participant, answer, sentence = [row[colname] for colname in ["occupation(0)", "other-participant(1)", "answer", "sentence"]]
  name = nmls[index]
  male_sent, female_sent, _ = generate(occupation, participant, answer, sentence, name)
  # index += 1
  # if index == 5:
  #   break
  male_ls.append(male_sent)
  female_ls.append(female_sent)

--2022-04-19 07:29:12--  https://raw.githubusercontent.com/rudinger/winogender-schemas/master/data/templates.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14903 (15K) [text/plain]
Saving to: ‘templates.tsv’

templates.tsv       100%[===================>]  14.55K  --.-KB/s    in 0s      

2022-04-19 07:29:12 (66.1 MB/s) - ‘templates.tsv’ saved [14903/14903]



### generate modifies conll03 testset

In [ ]:
nmls = ["Denali"]* 150

In [ ]:
# upload male.txt female.txt
conll03_male = []
conll03_female = []
with open("male.txt", "r") as f:
  conll03_male = f.read().splitlines()
for i in range(len(conll03_male)):
  conll03_male[i] = conll03_male[i].replace('$NAME', nmls[i])

with open("female.txt", "r") as f:
  conll03_female = f.read().splitlines()
for i in range(len(conll03_female)):
  conll03_female[i] = conll03_female[i].replace('$NAME', nmls[i])

# conll03_male
conll03_female

### Test for accurancy

In [ ]:
def accuracy(inputs):
  cor_cnt = 0
  nlprst = nlp(inputs)
  for sentence, names in zip(nlprst, nmls):
    nmtokens = tokenizer.tokenize(names)
    if len(nmtokens) == 1:
      for item in sentence:
        if item["word"] == nmtokens[0] and item["entity"] == 'B-PER':
          cor_cnt += 1
    elif len(nmtokens) > 1:
      for item in sentence:
        if item["word"] == nmtokens[0] and item["entity"] == 'B-PER':
          cor_cnt += 1/len(nmtokens)
        elif item["word"] in nmtokens[1:] and item["entity"] == 'I-PER':
          cor_cnt += 1/len(nmtokens)
  return cor_cnt

accuracy(female_ls)

82.5

In [ ]:
82.5/ len(female_ls)

0.6875

In [ ]:
98 / len(male_ls)

0.8166666666666667

In [ ]:
117.5 / len(conll03_male)

0.8514492753623188

In [ ]:
116.5/len(conll03_female)

0.8442028985507246

In [ ]:
# FInd sentences that contains gender info
from torch.utils.data import dataset
dataset = load_dataset('conll2003', split='test')
ids = set([])
for sts in dataset:
  for token in sts["tokens"]:
    if token in ["he", "him", "his", "she", "her", ""]:
      ids.add(sts["id"])
ids = list(ids)
# rst = nlp(" ".join(dataset[int(ids[0])]["tokens"]))
usf_ids = []
for item in ids:
  if 1 in dataset[int(item)]["ner_tags"]:
    usf_ids.append(int(item))
len(usf_ids)
with open("allgen.txt", "w") as f:
  for item in usf_ids:
    f.write( " ".join(dataset[item]["tokens"]) + "\n" )

### explain by the position of nlp results

In [ ]:
def explain_example(example):
  # example = male
  ner_results = nlp(example)
  entityList = findEntityPos(ner_results)
  explain_list = []
  weights_list = []

  for entityname in entityList:
    if entityList[entityname] != []:
      print(entityList[entityname])
      for entityls in entityList[entityname]:
        # print(entityname, entityls)
        entityls_0 = [item-1 for item in entityls]
        entitywrd = tokenizer.tokenize(example)[entityls_0[0]: entityls_0[-1]+1]
        # print(entitywrd)
        # entitywrd = " ".join(tmpentls)

        esampler = EntitySampler(
        replacement="[UNK]",
        max_replace=0.4,
        token_pattern=None,
        bow=False,
        entity_pos =entityls_0,
        entity_word = entitywrd
        )
        samples, similarity = esampler.sample_near(example, n_samples=5) ## hyperParam!!!
        # samples
        # samples_worlds= samples
        # print(samples)

        ep = EntityPrediction(posit = entityls)
        # prob = ep.pred_proba(samples_worlds)
        # print(prob)

        te = TestExplainer(
        n_samples=200,
        sampler=esampler,
        # sampler = None,
        # vec = vectorizer,
        position_dependent=False,
        random_state=42)

        if te.fit(example, ep.pred_proba) != None:
          explain = te.explain_prediction(
          target_names=["O", "MIS", "PER", "ORG", "LOC"],
          top_targets=3
          )
          weights = te.explain_weights(
          target_names=["O", "MIS", "PER", "ORG", "LOC"]
          )
          explain_list.append(explain)
          weights_list.append(weights)
  return explain_list, weights_list


### explain by a given name

In [ ]:
# Explanation pipeline:
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

def explain_example_wname(example, name):
  name_vec = tokenizer.tokenize(name)
  example_vec = tokenizer.tokenize(example)
  name_ind = []
  for elem in name_vec:
    name_ind.append(example_vec.index(elem)+1)
  # print("name_index", name_ind)
  entityls = name_ind
  explain_list = []
  weights_list = []

  # print(entityname, entityls)
  entityls_0 = [item-1 for item in entityls]
  entitywrd = tokenizer.tokenize(example)[entityls_0[0]: entityls_0[-1]+1]
  # print(entitywrd)

  esampler = EntitySampler(
  replacement="[UNK]",
  # max_replace=1.0,
  token_pattern=None,
  bow=False,
  entity_pos =entityls_0,
  entity_word = entitywrd
  )
  # Test if sampler works
  # samples, similarity = esampler.sample_near(example, n_samples=5) ## hyperParam!!!
  # print(samples)

  ep = EntityPrediction(posit = entityls)
  # Test if preditor works
  # prob = ep.pred_proba(samples)
  # print(prob)

  te = TestExplainer(
  n_samples=200,
  sampler=esampler,
  # sampler = None,
  # vec = vectorizer,
  position_dependent=False,
  random_state=42)

  # print("exp", example)
  if te.fit(example, ep.pred_proba) != None:
    explain = te.explain_prediction(
    target_names=["O", "MIS", "PER", "ORG", "LOC"],
    top_targets=3
    )
    weights = te.explain_weights(
    target_names=["O", "MIS", "PER", "ORG", "LOC"]
    )
    explain_list.append(explain)
    weights_list.append(weights)
  return explain_list, weights_list

# male = "Denali has been recovering from surgery on Monday which removed a small malignant tumour and half of her right lung ."
# els, wls = explain_example_wname(male, "Denali")

In [ ]:
exap =   "Kabecka's outburst three days ago has clearly turned some of the Australian public against her."

In [ ]:
els, wls = explain_example_wname(exap, "Kabecka")

In [ ]:
els[0]

Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7FA2C12878D0)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='PER', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='outburst', weight=1.2393979967480642, std=None, value=1.0), FeatureWeight(feature='ago has', weight=0.49615534690361973, std=None, value=1.0), FeatureWeight(feature='days', weight=0.35309771685160496, std=None, value=1.0), FeatureWeight(feature='kabecka s', weight=0.32885490224140346, std=None, value=1.0), FeatureWeight(feature='s', weight=0.32885490224140346, std=None, value=1.0), FeatureWeight(feature='the', weight=0.30202317142047014, std=None, value=1.0), FeatureWeight(feature='australian', weight=0.2617100888369367, std=None, value=1.0), FeatureWeight(feature='against', weight=0.23717290441353134, std=None, value=1.0), FeatureWeight(feature='clearly turned', weight=0.2077829944600651, std=None, value=1.0), FeatureWeight(feature='some', weight=0.18471114057647597, std=None, value=1.0), FeatureWeight(feature='three', weight=0.17838994234053795, std=None, value=1.0), FeatureWeight(feature='public', weight=0.1667968021080647, std=None, value=1.0), FeatureWeight(feature='ago', weight=0.1430170915212182, std=None, value=1.0), FeatureWeight(feature='of the', weight=0.1136075414806722, std=None, value=1.0), FeatureWeight(feature='of', weight=0.10625260136561678, std=None, value=1.0), FeatureWeight(feature='clearly', weight=0.09189012583146824, std=None, value=1.0), FeatureWeight(feature='turned some', weight=0.08203950292336729, std=None, value=1.0), FeatureWeight(feature='outburst three', weight=0.052696259001494015, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.03234289410847255, std=None, value=1.0), FeatureWeight(feature='public against', weight=0.03137312838435998, std=None, value=1.0)], neg=[FeatureWeight(feature='s outburst', weight=-1.07513243922881, std=None, value=1.0), FeatureWeight(feature='the australian', weight=-0.5166547323710621, std=None, value=1.0), FeatureWeight(feature='kabecka', weight=-0.45545412350470604, std=None, value=1.0), FeatureWeight(feature='against her', weight=-0.3973468729534952, std=None, value=1.0), FeatureWeight(feature='some of', weight=-0.37292443212387033, std=None, value=1.0), FeatureWeight(feature='australian public', weight=-0.25852523420983753, std=None, value=1.0), FeatureWeight(feature='has', weight=-0.2521527442698859, std=None, value=1.0), FeatureWeight(feature='her', weight=-0.2362481400148687, std=None, value=1.0), FeatureWeight(feature='three days', weight=-0.14084762507155588, std=None, value=1.0), FeatureWeight(feature='turned', weight=-0.10659074019659796, std=None, value=1.0), FeatureWeight(feature='days ago', weight=-0.014502959265514738, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.8183963518304167, score=1.1117870105486425, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document="kabecka's outburst three days ago has clearly turned some of the australian public against her.", spans=[('kabecka', [(0, 7)], -0.45545412350470604), ('s', [(8, 9)], 0.32885490224140346), ('outburst', [(10, 18)], 1.2393979967480642), ('three', [(19, 24)], 0.17838994234053795), ('days', [(25, 29)], 0.35309771685160496), ('ago', [(30, 33)], 0.1430170915212182), ('has', [(34, 37)], -0.2521527442698859), ('clearly', [(38, 45)], 0.09189012583146824), ('turned', [(46, 52)], -0.10659074019659796), ('some', [(53, 57)], 0.18471114057647597), ('of', [(58, 60)], 0.10625260136561678), ('the', [(61, 64)], 0.30202317142047014), ('australian', [(65, 75)], 0.2617100888369367), ('public', [(76, 82)], 0.1667968021080647), ('against', [(83, 90)], 0.23717290441353134), ('her', [(91, 94)], -0.2362481400148687), ('kabecka s', [(0, 7), (8, 9)], 0.32885490224140346), ('s outburst', [(8, 9), (10, 18)], -1.07513243922881), ('outburst three', [

In [ ]:
male_ls = conll03_male
female_ls = conll03_female
female_ls[24]

'General Denali saw rebel leader Laurent Kabila in Goma last week but the rebels told her the crisis was over because most of the Rwandan refugees have already gone home .'

In [ ]:
print([male_pos, male_neg, female_pos, female_neg]) #list len = 136
print([num / 136 for num in (male_pos, male_neg, female_pos, female_neg )])

[65.70701376881169, -3.7430303677742907, 35.28401093384387, -7.0422967275680675]
[0.4831398071236154, -0.02752228211598743, 0.259441256866499, -0.05178159358505932]


In [ ]:
female_pos, female_neg = 0, 0
for i in range(138):
  if i % 10 == 0:
    print(i)
  try:
    _, wls = explain_example_wname(female_ls[i], nmls[i])
    if len(wls) == 0:
      print("pass case %i" % i)
    else:
      for item in wls[0].targets[2].feature_weights.pos:
        if item.feature == "she" or item.feature == "her" or item.feature == "her" or item.feature == "herself":
          female_pos += item.weight
      for item in wls[0].targets[2].feature_weights.neg:
        if item.feature == "she" or item.feature == "her" or item.feature == "her" or item.feature == "herself":
          female_neg += item.weight
  except:
    print("pass case %i due to error" % i)

0
ERROR: unexplainable due to lack of negative samples
pass case 2
10
ERROR: unexplainable due to lack of negative samples
pass case 13
20
30
40
50
60
70
pass case 73 due to error
80
pass case 83 due to error
pass case 86 due to error
90
pass case 93 due to error
100
110
120
130


In [ ]:
male_pos, male_neg, female_pos, female_neg = 0, 0, 0, 0
for i in range(138):
  if i % 10 == 0:
    print(i)
  try:
    _, wls = explain_example_wname(male_ls[i], nmls[i])
    if len(wls) == 0:
      print("pass case %i" % i)
    else:
      for item in wls[0].targets[2].feature_weights.pos:
        if item.feature == "he" or item.feature == "him" or item.feature == "his"or item.feature == "himself":
          male_pos += item.weight
      for item in wls[0].targets[2].feature_weights.neg:
        if item.feature == "he" or item.feature == "him" or item.feature == "his"or item.feature == "himself":
          male_neg += item.weight
  except:
    print("pass case %i due to error" % i)

for i in range(138):
  if i % 10 == 0:
    print(i)
  _, wls = explain_example_wname(female_ls[i], nmls[i])
  if len(wls) == 0:
    print("pass case %i" % i)
  else:
    for item in wls[0].targets[2].feature_weights.pos:
      if item.feature == "she" or item.feature == "her" or item.feature == "her" or item.feature == "herself":
        female_pos += item.weight
    for item in wls[0].targets[2].feature_weights.neg:
      if item.feature == "she" or item.feature == "her" or item.feature == "her" or item.feature == "herself":
        female_neg += item.weight

np.array([male_pos, male_neg, female_pos, female_neg]) / 120

0
10
20
30
40
50
60
70
pass case 73 due to error
80
pass case 83 due to error
pass case 86 due to error
90
pass case 93 due to error
100
110
120
130
0
10
20
30
40
50


KeyboardInterrupt: ignored

In [ ]:
# len(explain_list)
explain_list[0]

Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7F97E1BA6E20)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='PER', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='customer', weight=1.6170811720146978, std=None, value=1.0), FeatureWeight(feature='customer yuxin', weight=1.6170811720146978, std=None, value=1.0), FeatureWeight(feature='the customer', weight=1.0622636306849211, std=None, value=1.0), FeatureWeight(feature='with', weight=0.9374282357071452, std=None, value=1.0), FeatureWeight(feature='told', weight=0.6499902736421361, std=None, value=1.0), FeatureWeight(feature='that', weight=0.45592387673823387, std=None, value=1.0), FeatureWeight(feature='yuxin that', weight=0.45592387673823387, std=None, value=1.0), FeatureWeight(feature='technician told', weight=0.3115518874664181, std=None, value=1.0), FeatureWeight(feature='pay with', weight=0.18318698611328102, std=None, value=1.0), FeatureWeight(feature='told the', weight=0.07837233529342053, std=None, value=1.0)], neg=[FeatureWeight(feature='the', weight=-1.0719976933673572, std=None, value=2.0), FeatureWeight(feature='cash', weight=-0.7770120496780175, std=None, value=1.0), FeatureWeight(feature='pay', weight=-0.6065512275399286, std=None, value=1.0), FeatureWeight(feature='could pay', weight=-0.473680008252595, std=None, value=1.0), FeatureWeight(feature='she', weight=-0.29454275342747155, std=None, value=1.0), FeatureWeight(feature='the technician', weight=-0.27906921557066783, std=None, value=1.0), FeatureWeight(feature='technician', weight=-0.17546094782143903, std=None, value=1.0), FeatureWeight(feature='she could', weight=-0.11141930759707795, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.001855895029303696, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.9917299411880405, score=3.5772143481293277, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='the technician told the customer yuxin that she could pay with cash.', spans=[('the', [(0, 3)], -1.0719976933673572), ('technician', [(4, 14)], -0.17546094782143903), ('told', [(15, 19)], 0.6499902736421361), ('the', [(20, 23)], -1.0719976933673572), ('customer', [(24, 32)], 1.6170811720146978), ('that', [(39, 43)], 0.45592387673823387), ('she', [(44, 47)], -0.29454275342747155), ('pay', [(54, 57)], -0.6065512275399286), ('with', [(58, 62)], 0.9374282357071452), ('cash', [(63, 67)], -0.7770120496780175), ('the technician', [(0, 3), (4, 14)], -0.27906921557066783), ('technician told', [(4, 14), (15, 19)], 0.3115518874664181), ('told the', [(15, 19), (20, 23)], 0.07837233529342053), ('the customer', [(20, 23), (24, 32)], 1.0622636306849211), ('customer yuxin', [(24, 32), (33, 38)], 1.6170811720146978), ('yuxin that', [(33, 38), (39, 43)], 0.45592387673823387), ('she could', [(44, 47), (48, 53)], -0.11141930759707795), ('could pay', [(48, 53), (54, 57)], -0.473680008252595), ('pay with', [(54, 57), (58, 62)], 0.18318698611328102)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=3.5790702431586308, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.001855895029303696, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='the', weight=0.595941064212001, std=None, value=2.0), FeatureWeight(feature='pay', weight=0.3866397150304273, std=None, value=1.0), FeatureWeight(feature='cash', weight=0.27580658837690847, std=None, value=1.0), FeatureWeight(feature='the technician', weight=0.12004740141311647, std=None, value=1.0), FeatureWeight(feature='could pay', weight=0.031291453603084295, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.0006159109650018162, std=None, value=

In [ ]:
# len(explain_list)
explain_list[1]

Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7F97E1BA6E20)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='PER', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='the customer', weight=1.8223678465224769, std=None, value=1.0), FeatureWeight(feature='customer', weight=1.3308112771561265, std=None, value=1.0), FeatureWeight(feature='customer yuxin', weight=1.3308112771561265, std=None, value=1.0), FeatureWeight(feature='the technician', weight=0.9317308358446906, std=None, value=1.0), FeatureWeight(feature='that he', weight=0.8400569126572242, std=None, value=1.0), FeatureWeight(feature='told', weight=0.8064787608078346, std=None, value=1.0), FeatureWeight(feature='that', weight=0.5746194265989416, std=None, value=1.0), FeatureWeight(feature='yuxin that', weight=0.5746194265989416, std=None, value=1.0), FeatureWeight(feature='technician told', weight=0.2860272433987942, std=None, value=1.0), FeatureWeight(feature='pay with', weight=0.0663890338996016, std=None, value=1.0), FeatureWeight(feature='pay', weight=0.03688653116000963, std=None, value=1.0)], neg=[FeatureWeight(feature='could', weight=-1.389459802151743, std=None, value=1.0), FeatureWeight(feature='technician', weight=-1.2685287247335415, std=None, value=1.0), FeatureWeight(feature='the', weight=-0.8826453673780594, std=None, value=2.0), FeatureWeight(feature='told the', weight=-0.8120021270946912, std=None, value=1.0), FeatureWeight(feature='yuxin', weight=-0.8054351796692167, std=None, value=1.0), FeatureWeight(feature='he', weight=-0.5025966331351488, std=None, value=1.0), FeatureWeight(feature='he could', weight=-0.4264541117730023, std=None, value=1.0), FeatureWeight(feature='could pay', weight=-0.16523732347149425, std=None, value=1.0), FeatureWeight(feature='cash', weight=-0.050584108612369616, std=None, value=1.0), FeatureWeight(feature='with', weight=-0.042815053819033506, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.032313840898724314, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.9212741304070767, score=2.222726299063743, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='the technician told the customer yuxin that he could pay with cash.', spans=[('the', [(0, 3)], -0.8826453673780594), ('technician', [(4, 14)], -1.2685287247335415), ('told', [(15, 19)], 0.8064787608078346), ('the', [(20, 23)], -0.8826453673780594), ('customer', [(24, 32)], 1.3308112771561265), ('yuxin', [(33, 38)], -0.8054351796692167), ('that', [(39, 43)], 0.5746194265989416), ('he', [(44, 46)], -0.5025966331351488), ('could', [(47, 52)], -1.389459802151743), ('pay', [(53, 56)], 0.03688653116000963), ('with', [(57, 61)], -0.042815053819033506), ('cash', [(62, 66)], -0.050584108612369616), ('the technician', [(0, 3), (4, 14)], 0.9317308358446906), ('technician told', [(4, 14), (15, 19)], 0.2860272433987942), ('told the', [(15, 19), (20, 23)], -0.8120021270946912), ('the customer', [(20, 23), (24, 32)], 1.8223678465224769), ('customer yuxin', [(24, 32), (33, 38)], 1.3308112771561265), ('yuxin that', [(33, 38), (39, 43)], 0.5746194265989416), ('that he', [(39, 43), (44, 46)], 0.8400569126572242), ('he could', [(44, 46), (47, 52)], -0.4264541117730023), ('could pay', [(47, 52), (53, 56)], -0.16523732347149425), ('pay with', [(53, 56), (57, 61)], 0.0663890338996016)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=2.2550401399624667, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.032313840898724314, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='could', weight=1.2751777152341643, std=None, value=1.0), FeatureWeight(feature='technicia

In [ ]:
# Test with tokenizer and sub-word Issue!
from transformers import DistilBertTokenizer

vectorizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

# vectorizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
ls = ['Sweet dreams are made of these, who am Dong to disagree, I travel the Utopia, Waang [UNK] looking for Something', 
      'Sweet dreams are made of these, who am Dong to disagree, I [UNK] [UNK] Utopia, Waang is looking for Something', 
      'Sweet dreams are [UNK] of these, who am Dong to disagree, I travel the Utopia, Waang is looking for Something', 
      'Sweet dreams are made of these, who am Dong to disagree, I travel [UNK] [UNK], Waang is looking for Something', 
      'Sweet dreams are made of these, who am Dong [UNK] disagree, I travel the [UNK], Waang is looking for Something']
      
# is_split_into_words
[ len(vectorizer.tokenize(ls[i])) for i in range(5)]
# tokenizer(ls, return_tensors="pt")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

[26, 26, 26, 25, 25]

In [ ]:
# from sklearn import train_test_split
# text = "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something"
# vectorizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

# te = TextExplainer(
# n_samples=10,
# sampler=esampler,
# # vec = vectorizer,
# position_dependent=False,
# random_state=42
# )

# (X_train, X_test,
#   similarity_train, similarity_test,
#   y_proba_train, y_proba_test) = train_test_split(samples,
#                                                   similarity,
#                                                   y_proba,)  

# te.fit(text, ep.pred_proba)

In [ ]:
# example = ["Sweet dreams are made of these, who am I to disagree, I travel the world and Sevensea, everybody is looking for Something",
#            "Sweet dreams are made of, who am I to disagree, I travel the world and Sevensea, everybody is looking for Something",
#            ]
example = "My name is Wolfgang and I live in Berlin"
# tokenizer(example)
tokenizer.unk_token

'[UNK]'

In [ ]:
# Test: try to get proba from model logits

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForTokenClassification.from_pretrained("bert-base-uncased")
# example = "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something"
example = ["Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something",
           "Sweet dreams are made of these, who am I to disagree, I [UNK] the Utopia, everybody is looking for Something",
           ]

inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)
loss = outputs.loss
logits = outputs.logits
logits
torch.set_printoptions(sci_mode=False)
softmax(logits.squeeze(0), dim=1)[:,17:19,:]
#note: tokenizer will add a starter token and a end token to the sentence.

tensor([[[    0.0000,     0.4313,     0.0066,     0.1401,     0.0129,
              0.0966,     0.0127,     0.9656,     0.0012],
         [    0.0000,     0.0208,     0.5156,     0.0126,     0.0599,
              0.0132,     0.1160,     0.0120,     0.9663]],

        [[    0.0000,     0.7311,     0.0072,     0.1209,     0.0161,
              0.0781,     0.0135,     0.6016,     0.0175],
         [    0.0000,     0.0202,     0.6036,     0.0304,     0.0326,
              0.0279,     0.0904,     0.0298,     0.2028]]],
       grad_fn=<SliceBackward0>)

In [ ]:
# generate sample but do not cover desired word
from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler
text = "Sweet dreams are made of these, who am I to disagree, I travel the Utopia, everybody is looking for Something"

sampler = MaskingTextSampler(
replacement="[UNK]",
max_replace=4,
token_pattern=None,
bow=False
)
samples, similarity = sampler.sample_near(text, n_samples=10)
badsample = []
for i in range(len(samples)):
  if tokenizer.tokenize(samples[i])[16:18] != ['U', '##topia']:
    badsample.append(i)

somelist = [i for j, i in enumerate(list(samples)) if j not in badsample]